In [9]:
import pandas as pd

In [10]:
# data source: https://github.com/emeeks/data_visualization_survey/tree/master/data
survey = pd.read_csv('cleaned_survey_results.csv')

In [11]:
survey.shape

(980, 44)

In [12]:
multi_q_dict = {"How is your organization using data visualization?": "How to use viz",
 "What level of data visualizations are used at your organization?": "What level of data",
 "Data visualization roles at your organization are found in what part of the organization?": "What part of org",
 "What technologies do you use to visualize data?": "Technologies", "How do you present your data visualizations?": "How to present", 
 "Which of these charts have you used in production in the last 6 months:": "Chart Types",
 "Who do you make data visualizations for?": "Who to make for","What do they use your data visualizations for?": "What to use for", 
 "Who do you look to as a thought leader in data visualization?": "Thought Leader"}

# Get data for multi-value field

In [13]:
def count_multi_value_q(df, time_col, real_col, time_col_name, real_col_name):
    new_df =  df[[time_col, real_col]]
    mask = (new_df[real_col].str.len() >1)
    new_df = new_df.loc[mask]
    new_df = pd.concat([pd.Series(row[time_col], row[real_col].split(';'))              
                        for _, row in new_df.iterrows()]).reset_index()
    new_df.columns = [real_col_name, time_col_name]
    new_df = new_df.groupby(real_col_name).size().reset_index().rename(columns={0:'count'}).sort_values(by='count', ascending=False).reset_index(drop=True)
    new_df
    new_df[real_col_name] = new_df[real_col_name].map(lambda x: '"' + str(x) + '"')
    new_df.columns = new_df.columns.map(lambda x: '"' + x + '"')
    
    new_df.to_csv("%s.csv"%real_col_name, index = False)
    return new_df

In [15]:
for key, value in multi_q_dict.iteritems():
    count_multi_value_q(survey, 'Timestamp', key, 'Timestamp', value)


# Get data for single value field

In [16]:
def count_single_value_q(df, real_col):
    new_df = df.groupby(real_col).size().reset_index().rename(columns={0:'count'}).sort_values(by='count', ascending=False).reset_index(drop=True)
    new_df[real_col] = new_df[real_col].map(lambda x: '"' + str(x) + '"')
    new_df.columns = new_df.columns.map(lambda x: '"' + x + '"')
    real_col = real_col.replace('?','').replace('/','')
    new_df.to_csv("%s.csv"%real_col, index = False)
    return new_df

In [17]:
multi_q_list = list(multi_q_dict.keys())

In [18]:
for i in survey.columns:
    if i not in multi_q_list:
        # print i
        count_single_value_q(survey, i)

# Process induvidual field with different requirement

In [19]:
def count_multi_value_details(df, time_col, real_col, time_col_name, real_col_name):
    new_df =  df[[time_col, real_col]]
    mask = (new_df[real_col].str.len() >1)
    new_df = new_df.loc[mask]
    new_df = pd.concat([pd.Series(row[time_col], row[real_col].split(';'))              
                        for _, row in new_df.iterrows()]).reset_index()
    new_df.columns = [real_col_name, time_col_name]
    test = new_df
    new_df = new_df.groupby(real_col_name).size().reset_index().rename(columns={0:'count'}).sort_values(by='count', ascending=False).reset_index(drop=True)
    new_df
    new_df[real_col_name] = new_df[real_col_name].map(lambda x: '"' + str(x) + '"')
    new_df.columns = new_df.columns.map(lambda x: '"' + x + '"')
    
    #new_df.to_csv("%s.csv"%real_col_name, index = False)
    return new_df, test


use_for, use_for_simple = count_multi_value_details(survey, "How many years have you been doing data visualization?", "What do they use your data visualizations for?", "Year", "Goal")
thought_leader, though_leader_simple = count_multi_value_details(survey, "How many years have you been doing data visualization?", "Who do you look to as a thought leader in data visualization?", "Year", "Thought leader")

In [20]:
use_for_simple = use_for_simple[use_for_simple.Goal.isin(['Analysis','Summarization','Research','Learning','Entertainment'])].groupby(['Year','Goal']).size().reset_index()

In [21]:
use_for_simple = use_for_simple.pivot(index='Year', columns='Goal', values=0).apply(lambda c: c / c.sum() * 100, axis=1).reset_index()

In [22]:
use_for_simple.rename(columns={'index':'Year'}, inplace=True)

In [23]:
use_for_simple['Year'] = use_for_simple.Year.map(lambda x: '"' + str(x) + '"')
use_for_simple.columns = use_for_simple.columns.map(lambda x: '"' + str(x) + '"')
use_for_simple.to_csv('groupby.tsv', index=None, sep='\t')

In [24]:
thought_leader.to_csv('thoughtleader.tsv', index=None, sep='\t')